# Pso problem for Homework 3

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Functions

In [2]:
def sphere(x):
    
    y = x[0] ** 2 + x[1] ** 2
    
    return y

def rosenbrock(x):
    
    y = 100 * ((x[1] - x[0] ** 2)) ** 2 + (x[0] -1) ** 2
    
    return y

def rastrigin(x):
    
    d = 2
    y = 10 * d;
    for i in range(d):
        y += x[i] ** 2 - 10 * np.cos(2 * np.pi * x[i])
        
    return y

def griewank(x):
    
    d = 2
    part1 = 0;
    for i in range(d):
        part1 += x[i] ** 2
    part2 = 1
    for i in range(d):
        part2 *= np.cos(x[i] / np.sqrt(i+1))
        
    y = 1 + part1 / 4000 - part2
    
    return y

## Class Particle

In [3]:
class Particle():
    
    def __init__(self, idn):
        self.id = idn
        self.position = np.zeros(2)
        self.velocity = np.zeros(2)
        self.pbest = np.zeros(2)
        self.gbest = np.zeros(2)
        
    def update_pbest(self, func):
        
        fitness = func(self.position)
        fbest = func(self.pbest)
        if fitness < fbest:
            self.pbest = self.position.copy()
            fbest = func(self.pbest)
        return fbest
        
    def update_velocity(self, c, v, **kw):
        
        if 'weight' in kw.keys():
            self.velocity = kw['weight'] * self.velocity + \
                c[0] * np.random.rand() * (self.pbest - self.position) + \
                c[1] * np.random.rand() * (self.gbest - self.position)
        elif 'constrictor' in kw.keys():
            self.velocity = kw['constrictor'] *(self.velocity + \
                c[0] * np.random.rand() * (self.pbest - self.position) + \
                c[1] * np.random.rand() * (self.gbest - self.position))
        else:
            self.velocity = self.velocity + \
                c[0] * np.random.rand() * (self.pbest - self.position) + \
                c[1] * np.random.rand() * (self.gbest - self.position)
        # limitition of velocity
        self.velocity[np.where(self.velocity > v[1])] = v[1]
        self.velocity[np.where(self.velocity < v[0])] = v[0]
    
        
    def update_position(self, x):
        self.position += self.velocity
        
        # limitation of position
        self.position[np.where(self.position > x[1])] = x[1]
        self.position[np.where(self.position < x[0])] = x[0]

## Class Pso

In [4]:
class Pso():
    
    def __init__(self,
                 dim = 2,
                 size = 25,
                 iters = 100,
                 func=sphere,
                 v = 1):
        self.dim = dim
        self.size = size    # number of particles
        self.iters = iters  # number of iterations
        self.func = func    # the fit function
        # Function space for each function
        if func == sphere or func == rastrigin:
            x = 5.12
        elif func == rosenbrock:
            x = 2.048
        elif func == griewank:
            x = 600
        self.x =(-x,x)      # limitation of position
        self.v = (-v,v)     # limitaion of velocity
        
    # Main interface
    def solution(self,c=(2, 2),top ='full',variant='main'):
        # Initial the particles
        particles = [Particle(i) for i in range(self.size)]
        for particle in particles:
            particle.position = np.random.rand(2) * (self.x[1]-self.x[0]) + self.x[0]
            particle.velocity = np.random.rand(2) * (self.v[1]-self.v[0]) + self.v[0]
            # Initial Pbest and Gbest
            particle.pbest = particle.position.copy()
        fbest = np.zeros(self.size)
        result = np.zeros(self.iters)
        # iterations
        for it in range(self.iters):
            
            for particle in particles:
                # update pbest for each particle
                fbest[particle.id] = particle.update_pbest(self.func)
        
            for particle in particles:
                # Update Gbest 
                if top == 'full':
                    pos = np.where(fbest == np.min(fbest))
                    particle.gbest = particles[pos[0][0]].pbest
                elif top == 'ring':
                    ## get neighbours id
                    if particle.id == self.size -1:
                        nearid = [particle.id-1,particle.id,0]
                    else:
                        nearid = [particle.id-1, particle.id, particle.id+1]
                    fnear = fbest[nearid]
                    pos = np.where(fnear == np.min(fnear))[0][0]
                    particle.gbest = particles[nearid[pos]].pbest
                elif top == '4-neighbours':
                    nearid = self.get_four_neighbers(particle.id)
                    fnear = fbest[nearid]
                    pos = np.where(fnear == np.min(fnear))[0][0]
                    particle.gbest = particles[nearid[pos]].pbest
                # Update velocity
                if variant == 'main':
                    particle.update_velocity(c, self.v)
                elif variant == 'weight':
                    ## weight is linear decreasing from 0.9 to 0.4
                    weight = 0.4 + 0.5 * (1 - it / self.iters)
                    particle.update_velocity(c, self.v, weight=weight)
                elif variant == 'constriction':
                    phi = 4.1
                    chi = 2 / np.abs(2 - phi - np.sqrt(phi ** 2 - 4 * phi))
                    particle.update_velocity(c, self.v,constrictor=chi)
                # update position
                particle.update_position(self.x);
            
            # save the best result in this iteration
            result[it] = np.min(fbest)
            
        return result
                
    def get_four_neighbers(self, index):
        # compute length of square
        r = int(np.sqrt(self.size))
        # generate a rxr square contains particle's id
        grid = np.arange(self.size).reshape(r,r)
        # compute row and column
        row = int(np.floor(index / r))
        column = int(index % r)
        
        # get the neighbers index
        ## up
        up = grid[row-1][column]
        ## down
        if row == r-1:
            down = grid[0][column]
        else:
            down = grid[row+1][column]
        ## left
        left = grid[row][column-1]
        ## right
        if column == r-1:
            right = grid[row][0]
        else:
            right = grid[row][column+1]
            
        return [up,down,left,right,index]

## Main Function

In [5]:
if __name__ == '__main__':
    
    fipairs = [(1,3),(2,2),(3,1)]
    topologies = ['full', 'ring', '4-neighbours']
    variants = ['main', 'weight', 'constriction']
    functions = {sphere:'sphere',
                 rosenbrock:'rosenbrock',
                 rastrigin:'rastrigin',
                 griewank:'griewank'}
    
    psoes = [Pso(func = func) for func in functions.keys()]
    for pso in psoes:
        for fipair in fipairs:
            for topology in topologies:
                for variant in variants:
                    best = []
                    for i in range(10):  
                        result = pso.solution(c=fipair,
                                              top=topology,
                                              variant=variant)
                        best.append(np.min(result))
                    print(f'function:{functions[pso.func]} |'
                          f'fipair:{fipair} |'
                          f'topology:{topology} |'
                          f'variant:{variant} |'
                          f'Error:{np.min(best)}')

function:sphere |fipair:(1, 3) |topology:full |variant:main |Error:5.111497207777128e-07
function:sphere |fipair:(1, 3) |topology:full |variant:weight |Error:4.156457993036057e-14
function:sphere |fipair:(1, 3) |topology:full |variant:constriction |Error:6.313792084723084e-15
function:sphere |fipair:(1, 3) |topology:ring |variant:main |Error:4.0066706146575445e-06
function:sphere |fipair:(1, 3) |topology:ring |variant:weight |Error:9.049767527265953e-12
function:sphere |fipair:(1, 3) |topology:ring |variant:constriction |Error:1.0381275836403549e-11
function:sphere |fipair:(1, 3) |topology:4-neighbours |variant:main |Error:1.1148783223324103e-05
function:sphere |fipair:(1, 3) |topology:4-neighbours |variant:weight |Error:3.001263335207124e-12
function:sphere |fipair:(1, 3) |topology:4-neighbours |variant:constriction |Error:1.3633394110912145e-12
function:sphere |fipair:(2, 2) |topology:full |variant:main |Error:7.8805916986217e-07
function:sphere |fipair:(2, 2) |topology:full |variant

function:griewank |fipair:(1, 3) |topology:ring |variant:weight |Error:0.08280830385355309
function:griewank |fipair:(1, 3) |topology:ring |variant:constriction |Error:0.17812454113697074
function:griewank |fipair:(1, 3) |topology:4-neighbours |variant:main |Error:0.07900221885846315
function:griewank |fipair:(1, 3) |topology:4-neighbours |variant:weight |Error:0.11971984708295635
function:griewank |fipair:(1, 3) |topology:4-neighbours |variant:constriction |Error:0.45980997170921023
function:griewank |fipair:(2, 2) |topology:full |variant:main |Error:0.4307473975421343
function:griewank |fipair:(2, 2) |topology:full |variant:weight |Error:0.346306071032108
function:griewank |fipair:(2, 2) |topology:full |variant:constriction |Error:0.31906376326659214
function:griewank |fipair:(2, 2) |topology:ring |variant:main |Error:0.6995655032723513
function:griewank |fipair:(2, 2) |topology:ring |variant:weight |Error:0.4444135315154184
function:griewank |fipair:(2, 2) |topology:ring |variant:co